# Invertebrate DNA metabarcoding data processing

I will use [metaBEAT](https://github.com/HullUni-bioinformatics/metaBEAT), a tool tailored towards reproducible and efficient analyses of metabarcoding data that was developed by Dr. Christoph Hahn (University of Graz) for the EvoHull group at University of Hull. The pipeline is still under active development and will likely be extended further in the future. The pipeline is available in a Docker container with all necessary dependencies. The Docker image builds on [ReproPhylo](https://github.com/HullUni-bioinformatics/ReproPhylo).

The data I will be analysing are COI sequences amplified from bulk tissue DNA extractions, or invertebrate soups, performed on sweep-net samples from ponds in Norfolk. The sweep-net samples were morphologically identified to lowest taxonomic level possible and size-sorted in three categories (S, M, L). From the lysate of these size categories during DNA extraction, 2 additional categories were created: unsorted (lysate pooled proportional to dry tissue weight of size categories) and sorted (lysate pooled proportional to number of specimens in each size category). Additionally, some specimens in net samples were individually preserved and extracted for the creation of DNA mock communities to investigate primer bias under different scenarios. An annealing temperature gradient was performed on the standardised low diversity and high diversity communities to investigate species drop-out as annealing temperature increases.

The metaBEAT tool is designed for complete bioinformatic analysis from raw data, and performs (optionally) de-multiplexing, quality filtering, chimera detection, clustering, and taxononomic assignment (outputs in `.biom` and `.tsv` formats). It currently supports BLAST, Kraken and phylogenetic placement (pplacer). Further approaches will be included in the future to allow for efficient and standardized comparative assessments of all approaches. A large number of options are offered at each step of the pipeline to tailor bioinformatic analysis to different projects.

##Data input

This notebook will perform basic processing (trimming, merging, chimera removal, clustering and taxonomic assignment) of the metabarcoding data.

Minimum input for an analysis is a set of query sequences in one or several files (a number of file formats are accepted, e.g. `.fasta`, `.fastq`). These will be run through the pipeline sequentially.

Information on the nature and location of the query sequence files must be provided in a separate tab-delimited text file via the -Q flags.

Each line in this text file should look as follows: unique sample_ID, format, file1, file2

The required text files can be generated in any text editor. So theoretically, nano could be used in the terminal to construct the text file. For reproducibility and ease, a simple program can be used to generate the required file.

In the cell below, it is produced using a simple python script. The script will list all files in the location to which you downloaded your Illumina data (specified via the 'datadir' variable). It assumes that there is a file ending in `_R1.fastq` for each sample. For each such file, it will extract the sample name from the filename and format the required line for the text file accordingly. The resulting file is called `Querymap.txt` (specified in the 'to' variable).

In [ ]:
!pwd

In [ ]:
!mkdir 1-trimming

In [ ]:
cd 1-trimming/

In [ ]:
!ls -1 ../raw_reads/

Prepare a text file specifying the samples to be processed, including the format and location of the reads - the querymap.

The next command expects two `.fastq` files (forward and reverse reads) per sample in the directory `../raw_reads/`. It expects the files to be named 'SampleID', followed by '_R1' or '_R2' to identify the forward/reverse read file respectively.

The raw data have been downloaded and demultiplexed. They can be found in `../raw_reads/`.

SampleID must correspond to the first column in a file called `Sample_accessions.tsv`. This will either be pre-made to correspond to downloading read data from the NCBI Sequence Read Archive, or you will have to make it. The marker is '12S'.

If the `Sample_accessions.tsv` was pre-made, use this code to proceed:

In [ ]:
#%%bash

#for a in $(cat ../Data/Sample_accessions.tsv | grep "12S" | cut -f 1 | grep "SampleID" -v)
#do
#    R1=$(ls -1 ../raw_reads/$a-12S_* | grep "_R1.fastq")
#    R2=$(ls -1 ../raw_reads/$a-12S_* | grep "_R2.fastq")
#
#    echo -e "$a\tfastq\t$R1\t$R2"
#done > Querymap.txt

In [ ]:
#!head -n 10 Querymap.txt

**OR...**

To make the `Sample_accessions.tsv` file, use:

In [ ]:
!echo "SampleID" > ../1-trimming/Sample_names.tsv

In [ ]:
%%bash

for a in $(ls ../raw_reads/ | grep "R1" | cut -d '_' -f 1)
do 
   SampleID=$a
   
   echo -e "$SampleID"
done >> ../1-trimming/Sample_names.tsv

In [ ]:
!cat ../1-trimming/Sample_names.tsv

In [ ]:
%%bash

for a in $(cat ../1-trimming/Sample_names.tsv | grep "SampleID" -v)
do
    R1=$(ls -1 ../raw_reads/$a* | grep "_R1")
    R2=$(ls -1  ../raw_reads/$a* | grep "_R2")

    echo -e "$a\tfastq\t$R1\t$R2"
done > Querymap.txt

In [ ]:
!head -n 4 Querymap.txt

To the `Querymap.txt` file, add two columns which specify the number of bases to remove from the forward and reverse read. In our case, we want to remove 26 bp to ensure that there is no forward or reverse primer left.

In [ ]:
%%bash

sed 's/$/&\t26/' Querymap.txt > Querymap_new.txt

Have a look (note that the output is probably line-wrapped):

In [ ]:
!head -n 4 Querymap_new.txt

In [ ]:
%%bash

sed 's/$/&\t26/' Querymap_new.txt > Querymap_final.txt

In [ ]:
!head -n 4 Querymap_final.txt

##Raw read processing

Now, perform basic quality trimming and clipping (Trimmomatic) and paired-end read merging (flash). metaBEAT will be used to process all 192 samples in one go.

In [ ]:
!metaBEAT_global.py -h

The amplicon is expected to be 313 bp long. With a readlength of 300 bp we don't expect to see any primer sequences, so it's not necessary to provide the primer sequence for the trimming algorithm.

Command to trim, remove adapter sequences, and merge reads:

In [ ]:
%%bash

metaBEAT_global.py \
-Q Querymap_final.txt \
--trim_qual 30 --read_crop 200 --trim_minlength 200 \
--merge --forward_only --product_length 313 \
-@ L.Harper@2015.hull.ac.uk \
-n 5 -v &> log

Read processing will take several hours.


# Visualise query survival after trimming

metaBEAT will generate a directory with all temporary files that were created during the processing for each sample and will record useful stats summarizing the data processing in the file `metaBEAT_read_stats.csv`. You can explore the table manually or quickly plot out some of these stats here:

In [ ]:
%matplotlib inline
import pandas as pd

df = pd.read_csv('metaBEAT_read_stats.csv',index_col=0)
df['fraction'] = df['queries']/(df['total']*0.5)
df.fraction.hist(bins=50)

Detailed information on what metaBEAT did to each sample is contained in the `log` file. It contains the exact commands that were run for each sample during each step of the process.

In [ ]:
!tail -n 100 log

The next steps in the processing will be chimera detection, and global clustering of the centroids from all clusters from all samples to produce denovo OTUs. The temporary files from the global clustering and the final OTU table were written to the directory `./GLOBAL`.

In [ ]:
!ls GLOBAL/

The denovo OTU table (numbers are reads) can be viewed to see how OTUs are distributed across your samples. 


# Chimera detection

Some stats on the read counts before/after trimming, merging etc. are summarised for you in `metaBEAT_read_stats.csv`.

Next stage of the processing is chimera detection and removal of putative chimeric sequences. We'll do that using uchime as implemented in vsearch.

In [ ]:
!pwd

In [ ]:
cd ../

In [ ]:
!mkdir 2-chimera_detection

In [ ]:
cd 2-chimera_detection

Convert reference database from GenBank to fasta format to be used in chimera detection.

Prepare `REFmap.txt` file, i.e. text file that specifies the location and the format of the reference to be used.
The reference sequences in GenBank format are present in subdirectories for each vertebrate group in the `../reference_database` directory.

In [ ]:
!ls ../reference_database/

In [ ]:
!echo '../reference_database/CO1_Acaria_Aracnida_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Annelida_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Coleoptera_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Crustacea_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_EPNM_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Hemiptera-Hymenoptera_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Mollusca_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Odonata_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Trichoptera_Lepidoptera_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/AssassinBug.fasta\tfasta'> REFmap.txt

In [ ]:
!cat REFmap.txt

In [ ]:
!metaBEAT_global.py -h

In [ ]:
%%bash

metaBEAT_global.py \
-R REFmap.txt \
-f \
-@ L.Harper@2015.hull.ac.uk

This will produce `refs.fasta`.

In [ ]:
!head refs.fasta

Now run chimera detection.

In [ ]:
%%bash


for a in $(cut -f 1 ../1-trimming/Querymap.txt)
do
    if [ -s ../1-trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../1-trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log 
        cd ..

    else
        echo -e "$a is empty"
    fi
done

In [ ]:
!tail CAKE-L-01/log

In [ ]:
!tail WRONG-UN-03/log

# Clustering and taxonomic assignment

In [ ]:
!pwd

In [ ]:
cd ..

In [ ]:
!mkdir 3-taxonomic_assignment

In [ ]:
cd 3-taxonomic_assignment/

Produce the text file containing the reference sequences using the command line - we call it `REFmap.txt`.

In [ ]:
!echo '../reference_database/CO1_Acaria_Aracnida_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Annelida_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Coleoptera_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Crustacea_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_EPNM_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Hemiptera-Hymenoptera_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Mollusca_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Odonata_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/CO1_Trichoptera_Lepidoptera_SATIVA_cleaned.gb\tgb\n' \
'../reference_database/AssassinBug.fasta\tfasta'> REFmap.txt

In [ ]:
!cat REFmap.txt

Produce the text file containing non-chimera query sequences - `Querymap.txt`.

In [ ]:
%%bash

#Querymap
for a in $(ls -l ../2-chimera_detection/ | grep "^d" | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    echo -e "$a-nc\tfasta\t../2-chimera_detection/$a/$a-nonchimeras.fasta"
done > Querymap.txt

In [ ]:
!cat Querymap.txt

The Querymap.txt file has been made but includes the `./GLOBAL` directory in which all centroids and queries are contained. This will cause metaBEAT to fail so it must be removed manually from the `Querymap.txt` file.

In [ ]:
!sed '/GLOBAL/d' Querymap.txt > Querymap_final.txt

In [ ]:
!cat Querymap_final.txt

The metaBEAT image comes prepared with a database that contains taxonomic information for taxids. However, new taxids are constantly being added to Genbank, and the database (it's stored in the image in `/usr/bin/taxonomy.db`) contained in the image is not necessarily up to date. If the above command gives you errors about certain taxids not being found in the database you can try to update the database using code below. This will take a few minutes, depending on your internet connection.

In [ ]:
!taxit new_database /usr/bin/taxonomy.db

That's almost it. Now start the pipeline to do sequence clustering and taxonomic assignment of non-chimera queries via metaBEAT. As input, `Querymap.txt` containing samples that have been trimmed, merged and checked for chimeras, and the `REFmap.txt` file must be specified. metaBEAT will be asked to attempt taxonomic assignment using BLAST.

metaBEAT will automatically wrangle the data into the particular file formats that are required by each of the methods, run all necessary steps, and finally convert the outputs of each program to a standardized BIOM table.

GO!

In [ ]:
!metaBEAT_global.py -h

In [ ]:
%%bash

metaBEAT_global.py \
-Q Querymap_final.txt \
-R REFmap.txt \
--cluster --clust_match 0.97 --clust_cov 3 \
--blast --min_ident 0.90 --min_ali_length 0.8 \
-m COI -n 5 \
-E -v \
-@ L.Harper@2015.hull.ac.uk \
-o COI-trim30_crop200_merge-forwonly_nonchimera_c0.97cov3_blast90 &> log_blast90

In [ ]:
!tail -n 50 log_blast90

metaBEAT will generate a directory with all temporary files that were created during the processing for each sample and will record useful stats summarizing the data processing in the file `CO1-trim30_crop200_merge-forwonly_nonchimera_c0.97cov3_blast90.csv

Note that the entire process only took ~10 minutes.

All BLAST and KRAKEN outputs are kept in the directory ./GLOBAL.
metaBEAT has produced results tables in BIOM format for each of the approaches in the corresponding directory. A nice tool for interactive exploration of results is phinch. Load one of the tables (*.biom) and see if you can get an overview of the results.

# Unassigned reads

It is important to know the identity of unassigned reads for contamination or reference database ambiguities. This code will BLAST unassigned reads against the entirety of GenBank.

Based on the results of the 90% BLAST identity metaBEAT run, a new BIOM table containing only OTUs that were not taxonomically assigned is generated. A fasta file with the corresponding sequences is also prepared.

Required files:
- fasta file containing all query sequences (global centroids), as produced by 90% identity metaBEAT run
- taxonomy annotated OTU biom table in json format from a metaBEAT run. Not the taxonomy collapsed BIOM table.

Load the necessary functions. Functions are in place as of version '0.97.4-global' (commit: 9110e5a3f4a979e85733f83cb0388b00586544f6).

In [ ]:
!pwd

In [ ]:
cd ..

In [ ]:
!mkdir 4-unassigned

In [ ]:
cd 4-unassigned/

In [ ]:
import metaBEAT_global_misc_functions as mb

In [ ]:
table = mb.load_BIOM('../3-taxonomic_assignment/GLOBAL/BLAST_0.9/COI-trim30_crop200_merge-forwonly_nonchimera_c0.97cov3_blast90-OTU-taxonomy.blast.biom', 
                     informat='json')

In [ ]:
# double check that we've got a table
print table

Extract only OTUs that were not assigned to **any** taxonomic level, i.e. true 'unassigned'.

In [ ]:
unassigned_table = mb.BIOM_return_by_tax_level(taxlevel='unassigned', BIOM=table, invert=False)

In [ ]:
# check metadata in new table to see if we only got the unassigned bits
print unassigned_table.metadata(axis='observation')

In [ ]:
!ls ../3-taxonomic_assignment/GLOBAL/

Extract only sequences mentioned in the table.

In [ ]:
mb.extract_fasta_by_BIOM_OTU_ids(in_fasta='../3-taxonomic_assignment/GLOBAL/global_queries.fasta', 
                                 BIOM=unassigned_table, 
                                 out_fasta='../3-taxonomic_assignment/GLOBAL/unassigned_only.fasta')

In [ ]:
!ls ../3-taxonomic_assignment/GLOBAL/

In [ ]:
unassigned_table_notax = mb.drop_BIOM_taxonomy(unassigned_table)

In [ ]:
print unassigned_table_notax.metadata(axis='observation')

In [ ]:
mb.write_BIOM(BIOM=unassigned_table_notax, target_prefix='../3-taxonomic_assignment/GLOBAL/unassigned_only_denovo', outfmt=['json','tsv'])

In [ ]:
!ls ../3-taxonomic_assignment/GLOBAL

In [ ]:
!mv ../3-taxonomic_assignment/GLOBAL/u* ../4-unassigned/

In [ ]:
!pwd

In [ ]:
!ls ../4-unassigned/

When running the unassigned BLAST, metaBEAT may get stuck for a number of reasons:
- the `gb_to_taxid.csv` file may be missing taxids, in which case you have to fetch manually from GenBank
- metaBEAT may not be able to fetch a taxid from GenBank because a record belongs to a different public database, e.g. EMBL, protein database
- the NCBI taxonomy stored in the current metaBEAT image may be out of date

The command below will update the `metaBEAT_global.py` script locally so that it accepts records from the most common public sequence databases.

In [ ]:
!sed -i "s/for tag in \['gb','dbj']/for tag in \['gb','dbj','emb','pdb']/g" /usr/bin/metaBEAT_global.py

The command below will update the NCBI taxonomy database locally in the metaBEAT image.

In [ ]:
!taxit new_database /usr/bin/taxonomy.db

In [ ]:
%%bash

metaBEAT_global.py \
-B unassigned_only_denovo.biom \
--g_queries unassigned_only.fasta \
--cluster --clust_match 0.97 --clust_cov 3 \
--blast --blast_db ../../entireGB_DB/nt-Feb18/nt --min_ident 0.90 --min_ali_length 0.8 \
-m COI -n 5 \
-E -v \
-@ L.Harper@2015.hull.ac.uk \
-o COI-trim30_crop200_merge-forwonly_nonchimera_cm0.97cov3_blast100_unassigned &> log_unassigned

In [ ]:
!tail -100 log_unassigned

If you encounter the other problem of the `gb_to_taxid.csv` file missing taxids, follow these steps to manually add the problematic taxids:
- In the log file, go to the last query metaBEAT was processing when it broke
- Count the number of hits that were successfully processed, the last hit is the one that broke the pipeline. For example, taxids were successfully fetched/seen before for 11 hits, it is the 12th hit that broke the pipeline.
- Copy the name of the query.
- In a terminal, navigate to the directory `../4-unassigned/GLOBAL/BLAST_0.98/`
- Using vim, open the `global_blastn.out.xml` file.
- Enter '/' and paste the name of the problematic query, then hit enter.
- When the query is displayed, go to the hit that broke the pipeline.
- Copy the gi number for that hit, then search NCBI nucleotide database with it in a web browser.
- When the associated record is displayed, click on the organism for that record to go to the NCBI taxonomy page.
- Now open the `gb_to_taxid.csv` file in the directory `../4-unassigned`, and append the gi number and taxid for the problematic hit (**NB: you may need to move the `gb_to_taxid.csv` file to a different location to edit and save changes, then move back into the directory `../4-unassigned`**).
- Finally, in the terminal vim is open, enter ':q!' to exit without saving changes. 

You can now re-run metaBEAT.